# Step 1: data format converting

Define the fields we want to extract (total of 21 fields in order)
According to the document - https://csr.lanl.gov/data/2017/#citing
Need to review the content of the file to see if some attributes should be ignored.
   1. 'Time',
   2. 'EventID',
   3. 'LogHost',
   4. 'LogonType',
   5. 'LogonTypeDescription',
   6. 'UserName',
   7. 'DomainName',
   8. 'LogonID',
   9. 'SubjectUserName',
  10. 'SubjectDomainName',
  11. 'SubjectLogonID',
  12. 'Status',
  13. 'Source',
  14. 'ServiceName',
  15. 'Destination',
  16. 'AuthenticationPackage',
  17. 'FailureReason',
  18. 'ProcessName',
  19. 'ProcessID',
  20. 'ParentProcessName',
  21. 'ParentProcessID'


# Step 2: Data loading into dataframe

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Unable to process the whole datafile in a dataframe
# divide the original file into 3 csv files (sequentially instead of random shuffle)
# unable to use the shuffle function due to the memory limitation
day1Part1 = pd.read_csv('01R_part1.csv')
print(day1Part1.dtypes)
print(day1Part1.head())

print("************day01 Part 1 stat report***********")
size = len(day1Part1)
print(day1Part1.shape)
print(day1Part1.describe())
print(" ====the number of NULL for each column====== ")
print(day1Part1.isnull().sum())
print(" ====null ratio for each column====== ")
print(day1Part1.isnull().sum()/size)
print(" ====the number of NON-NULL values for each column====== ")
print(size - day1Part1.isnull().sum())

UserName             object
EventID               int64
LogHost              object
LogonID              object
DomainName           object
ParentProcessName    object
ParentProcessID      object
ProcessName          object
Time                  int64
ProcessID            object
dtype: object
      UserName  EventID     LogHost LogonID DomainName ParentProcessName   
0  Comp991643$     4688  Comp991643   0x3e7  Domain001          services  \
1  Comp736087$     4688  Comp736087   0x3e7  Domain001          services   
2  Comp006850$     4688  Comp006850   0x3e7  Domain001          services   
3  Comp466209$     4688  Comp466209   0x3e7  Domain001          services   
4  Comp688526$     4688  Comp688526   0x3e7  Domain001          services   

  ParentProcessID   ProcessName  Time ProcessID  
0           0x334  rundll32.exe     1     0xc0c  
1           0x2e8   svchost.exe     1    0x2074  
2           0x278   svchost.exe     1     0x498  
3           0x354     vssvc.exe     1    0x2d20  

In [2]:
day1Part2 = pd.read_csv('01R_part2.csv')
print(day1Part2.dtypes)
print(day1Part2.head())

print("************day01 Part 2 stat report***********")
size = len(day1Part2)
print(day1Part2.shape)
print(day1Part2.describe())
print(" ====the number of NULL for each column====== ")
print(day1Part2.isnull().sum())
print(" ====null ratio for each column====== ")
print(day1Part2.isnull().sum()/size)
print(" ====the number of NON-NULL values for each column====== ")
print(size - day1Part2.isnull().sum())

UserName             object
EventID               int64
LogHost              object
LogonID              object
DomainName           object
ParentProcessName    object
ParentProcessID      object
ProcessName          object
Time                  int64
ProcessID            object
dtype: object
      UserName  EventID     LogHost    LogonID    DomainName   
0       system     4624  Comp828729      0x3e7  nt authority  \
1       system     4672  Comp828729      0x3e7  nt authority   
2       system     4672  Comp423597      0x3e7  nt authority   
3  Comp107561$     4688  Comp107561      0x3e7     Domain001   
4  Comp939275$     4624  Comp661433  0x23a8696     Domain001   

  ParentProcessName ParentProcessID   ProcessName  Time ProcessID  
0               NaN             NaN  services.exe     1     0x29c  
1               NaN             NaN           NaN     1       NaN  
2               NaN             NaN           NaN     1       NaN  
3          services           0x2cc  rundll32.exe

In [ ]:
day1Part3 = pd.read_csv('01R_part3.csv')
print(day1Part3.dtypes)
print(day1Part3.head())

print("************day01 Part 3 stat report***********")
size = len(day1Part3)
print(day1Part3.shape)
print(day1Part3.describe())
print(" ====the number of NULL for each column====== ")
print(day1Part3.isnull().sum())
print(" ====null ratio for each column====== ")
print(day1Part3.isnull().sum()/size)
print(" ====the number of NON-NULL values for each column====== ")
print(size - day1Part3.isnull().sum())

[[summary]]

1. Look like the data entries sorted by "Time".
2. Less than 2% data entries with values in SubjectXXXX (UserName, DomainName and LogonID)columns. However it may still have enough data entries (~900K total).
3. Is it fine to divide the datafile in a sequential order for the further investigation?
4. Can we delete some columns not used for the key features to reduce the file size of the origional datafile?

# Key Features and Calculations

[[ Priority 1 Feature ]] 
1. Event Frequency per User - Helps establish a baseline of normal user activity levels.
   Count the occurrences of each EventID per user over a defined time window daily
2. Successful vs. Failed Logins Ratio - Helps detect credential stuffing, brute force attacks, or password guessing attempts.
   Calculate the ratio of successful logins (4624) to failed logins (4625).


In [5]:
# Event Frequency per User
day1Part1.groupby(["UserName", "EventID", "Time"]).size().reset_index(name="EventCount")


,UserName,EventID,Time,EventCount
0,ActiveDirectory$,4624,2,2
1,ActiveDirectory$,4624,4,3
2,ActiveDirectory$,4624,5,4
3,ActiveDirectory$,4624,6,1
4,ActiveDirectory$,4624,8,3
...,...,...,...,...
10025651,winservice,4769,82811,1
10025652,winservice,4769,82821,1
10025653,winservice,4769,84605,1
10025654,winservice,4776,75604,1


In [4]:
day1Part2.groupby(["UserName", "EventID", "Time"]).size().reset_index(name="EventCount")

,UserName,EventID,Time,EventCount
0,ActiveDirectory$,4624,2,1
1,ActiveDirectory$,4624,4,7
2,ActiveDirectory$,4624,5,6
3,ActiveDirectory$,4624,6,2
4,ActiveDirectory$,4624,7,1
...,...,...,...,...
10024365,winservice,4769,84600,1
10024366,winservice,4769,84605,1
10024367,winservice,4776,75606,1
10024368,winservice,4776,75625,1


In [ ]:
day1Part3.groupby(["UserName", "EventID", "Time"]).size().reset_index(name="EventCount")

In [6]:
# uccessful vs. Failed Logins Ratio
success = day1Part1[day1Part1["EventID"] == 4624].groupby("UserName").size()
print(f"the number of null in the success table {success.isnull().sum()}")
success

the number of null in the success table 0


UserName
ActiveDirectory$     93230
Administrator       119704
Anonymous            79686
AppService          307111
Comp000082$            105
                     ...  
User999959              91
local service          482
network service        524
system               62651
winservice            3936
Length: 17462, dtype: int64

In [7]:
fail = day1Part1[day1Part1["EventID"] == 4625].groupby("UserName").size()
print(f"the number of null in the fail table {fail.isnull().sum()}")
fail

the number of null in the fail table 0


UserName
Administrator    15053
AppService       14677
Comp032744$         15
Comp041781$          1
Comp053876$         17
                 ...  
User995456           1
User997760           2
User998338           2
User998627           1
User999918           1
Length: 885, dtype: int64

In [8]:
logon_ratio = success / (success + fail + 1)  # Avoid division by zero
print(f"the number of null in the logon_ratio table {logon_ratio.isnull().sum()} - either missing success or fail ")
logon_ratio

the number of null in the logon_ratio table 16733 - either missing success or fail 


UserName
ActiveDirectory$         NaN
Administrator       0.888289
Anonymous                NaN
AppService          0.954386
Comp000082$              NaN
                      ...   
User999959               NaN
local service            NaN
network service          NaN
system                   NaN
winservice               NaN
Length: 17540, dtype: float64

In [ ]:
#day1Part2

In [ ]:
#day1Part3

[[ Priority 2 Feature ]] 

3. Session Duration per User -  Identify short-lived or abnormally long sessions.
   Calculate the time difference between login (4624) and logoff (4634) events per session.
4. Workstation Lock/Unlock Behavior - Detects anomalies in workstation activity (e.g., frequent screen locking/unlocking).
   Compute the time interval between workstation lock (4800) and unlock (4801) events.
5. Process Start and End Activity - Helps detect unusual process executions (e.g., malware launching processes).
   Count the number of process start (4688) and process end (4689) events per user.


In [9]:
# Session Duration per User
sessionDuration = day1Part1[day1Part1["EventID"].isin([4624, 4634])].groupby("LogonID")["Time"].agg(["min", "max"]).reset_index()
# Add one more column to see the difference between min/max?
sessionDuration['diff'] = sessionDuration['max'] - sessionDuration['min']
print(f"Diff is NOT ZERO - {sum(sessionDuration['diff'] != 0 )}")
sessionDuration


Diff is NOT ZERO - 886000


,LogonID,min,max,diff
0,0x100000d7,25494,25494,0
1,0x10000105,25494,25494,0
2,0x10000135,25494,25494,0
3,0x1000015d,25494,25494,0
4,0x100001d2,25494,25494,0
...,...,...,...,...
7642770,0xffffa322,38772,38772,0
7642771,0xffffa3c,43676,43690,14
7642772,0xffffd1e,25516,25516,0
7642773,0xfffff30,25494,25494,0


In [10]:
# Workstation Lock/Unlock Behavior
lockBehavior = day1Part1[day1Part1["EventID"].isin([4800, 4801])].groupby("UserName")["Time"].diff()
print(lockBehavior.isnull().sum())
lockBehavior

3956


19961           NaN
57299           NaN
108001          NaN
228167          NaN
340118          NaN
             ...   
17862747     7742.0
18005952     9460.0
18019841    10728.0
18310082    34318.0
18516790     8500.0
Name: Time, Length: 10312, dtype: float64

In [11]:
# Process Start and End Activity
process_counts = day1Part1[day1Part1["EventID"].isin([4688, 4689])].groupby(["UserName", "EventID"]).size().unstack()
print(process_counts.isnull().sum())
process_counts


EventID
4688        0
4689    15948
dtype: int64


EventID,4688,4689
UserName,,
ActiveDirectory$,1085.0,NaN
Administrator,934.0,NaN
AppService,13037.0,NaN
Comp000082$,252.0,NaN
Comp000116$,325.0,NaN
...,...,...
User999918,7.0,NaN
User999948,13.0,NaN
User999959,33.0,NaN


In [12]:
result = process_counts["ProcessRatio"] = process_counts[4688] / (process_counts[4689] + 1)  # Avoid div by zero
print(result.isnull().sum())
result

15948


UserName
ActiveDirectory$            NaN
Administrator               NaN
AppService                  NaN
Comp000082$                 NaN
Comp000116$                 NaN
                       ...     
User999918                  NaN
User999948                  NaN
User999959                  NaN
local service       1523.326531
winservice                  NaN
Length: 15952, dtype: float64